## Calculate coverage perbase for input and output

**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ASSAY=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
ls ${FD_INP}

KS91_K562_ASTARRseq.Input.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.WGS.unstranded.bed.gz
KS91_K562_AST

In [3]:
ASSAY=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count

FP_INPS=($(ls ${FD_INP}/*SUBSET*))
echo "Total:  ${#FP_INPS[@]}"
basename -a ${FP_INPS[@]}

Total:  10
KS91_K562_ASTARRseq.Input.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.SUBSET.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep4.SUBSET.unstranded.bed.gz


## RUN

In [4]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/coverage_perbase_subset_ASTARR.%a.txt \
    --array 0-9 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=KS91_K562_ASTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_counts
FD_REG=${FD_RES}/results/region
FP_REG=${FD_REG}/region_screened_selected_perbase.bed.gz

FP_INPS=($(ls ${FD_INP}/*SUBSET*))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_REG}
echo
echo "show first few lines of file"
zcat ${FP_REG} | head -5
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_INP}
echo
echo "show first few lines of file"
zcat ${FP_INP} | head -5
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o sum \
| gzip -c \
> ${FP_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29574548


## Check

In [5]:
cat ${FD_LOG}/coverage_perbase_subset_ASTARR.0.txt

Hostname:           x3-01-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-15-23+13:38:24

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected_perbase.bed.gz

show first few lines of file
chr11	4000000	4000001
chr11	4000001	4000002
chr11	4000002	4000003
chr11	4000003	4000004
chr11	4000004	4000005

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep1.SUBSET.unstranded.bed.gz

show first few lines of file
chr11	3999592	4000108	chr11_3999592_4000108	1	.	chr11	4000000	6600000	chr11:4091884-6505900	108
chr11	3999629	4000121	chr11_3999629_4000121	1	.	chr11	4000000	6600000	chr11:4091884-6505900	121
chr11	3999670	4000006	chr11_3999670_4000006	1	.	chr11	4000000	6600000	chr11:4091884-6505900	6
chr11	3999679	4000092	chr11_3999679_4000092	1	.	chr11	4000000	6600000	chr11:4091884-6505

In [6]:
cat ${FD_LOG}/coverage_perbase_subset_ASTARR.9.txt

Hostname:           x3-04-4.genome.duke.edu
Slurm Array Index:  9
Time Stamp:         03-15-23+13:38:25

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected_perbase.bed.gz

show first few lines of file
chr11	4000000	4000001
chr11	4000001	4000002
chr11	4000002	4000003
chr11	4000003	4000004
chr11	4000004	4000005

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Output.rep4.SUBSET.unstranded.bed.gz

show first few lines of file
chr11	3999534	4000055	chr11_3999534_4000055	1	.	chr11	4000000	6600000	chr11:4091884-6505900	55
chr11	3999534	4000056	chr11_3999534_4000056	1	.	chr11	4000000	6600000	chr11:4091884-6505900	56
chr11	3999535	4000056	chr11_3999535_4000056	1	.	chr11	4000000	6600000	chr11:4091884-6505900	56
chr11	3999569	4000150	chr11_3999569_4000150	1	.	chr11	4000000	6600000	chr11:4091884-6505